# 각 게임에 대하여 핵심 키워드 (광고, 접속, 현질, 재미)에 대한 각각의 감성 점수 내기

## 6. 키워드별 감성분석

In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 1.3 MB/s 
     |████████████████████████████████| 86 kB 5.5 MB/s 
     |████████████████████████████████| 448 kB 57.1 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install fasttext

     |████████████████████████████████| 68 kB 2.9 MB/s 
  Using cached pybind11-2.7.1-py2.py3-none-any.whl (200 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3095238 sha256=a80f183556fc5c01677a3095577f73214da52524c86205a7d978eea1668e95b2
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [ ]:
#import fasttext
import csv
import pandas as pd
from sklearn.model_selection import train_test_split
#from konlpy.tag import Okt

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/munning_rachine/dataset/final.csv')

In [ ]:
df = df.drop(['Collection Date', 'Comment Date', 'User', 'Helpful', 'Answer','Comment_processed', 'Comment_cleansing'], axis=1)
df

,App Name,Rating,Comment,split,advertisement,connect,buying,fun
0,오딘: 발할라 라이징,1,첫날에 결제를 하고 꾸준히 소소하게 즐기려고 했으나 대기열이 많아 접속자체를 할수가...,첫날에 결제를 하고 꾸준히 대기 열이 많아 접속 자체를 할 수가 없습니다.,0,1,1,0
1,오딘: 발할라 라이징,1,첫날에 결제를 하고 꾸준히 소소하게 즐기려고 했으나 대기열이 많아 접속자체를 할수가...,돈은 썼는데 게임을 하다가 질려서 접은 것도 아니고 접속이 돼야 무슨 게임을 하죠,0,1,0,0
2,오딘: 발할라 라이징,1,첫날에 결제를 하고 꾸준히 소소하게 즐기려고 했으나 대기열이 많아 접속자체를 할수가...,하루 종일 대기 열기 다리느라 핸드폰을 켜 둘 수도 없고 직장생활하며 집에서 잠깐씩...,0,1,0,0
3,오딘: 발할라 라이징,1,첫날에 결제를 하고 꾸준히 소소하게 즐기려고 했으나 대기열이 많아 접속자체를 할수가...,대기 열을 계속 이대로 둘 거면 환불해주시기 바랍니다.,0,0,0,0
4,오딘: 발할라 라이징,1,모바일 게임 그동안 참 많이 했는데... 다른 게임들은 그나마 실행후 플레이 하면서...,모바일 게임 그동안 참 많이 했는데...,0,0,0,0
...,...,...,...,...,...,...,...,...
33176,Rider,1,응 개똥게임 차선택이안된고클릭도 안되,응 개똥 게임 찬 선택이 안된고 클릭도 안돼,0,1,0,0
33177,Rider,5,눈동자 지 너무 잘 먹고,눈동자 지 너무 잘 먹고,0,0,0,0
33178,Rider,5,그냥그대로멈추는데? 와 버그대박 불상해서 많ㅇㄱ드릴게요,그냥 그대로 멈추는데?,0,0,0,0
33179,Rider,5,그냥그대로멈추는데? 와 버그대박 불상해서 많ㅇㄱ드릴게요,와 버그 대박 불상해서 많ㅇㄱ드릴게요,0,1,0,0


In [ ]:
okt = Okt()
def tokenizer(dataframe, col_name):
  for (i, row) in enumerate(dataframe[col_name]):
    v = ''
    adj = ''
    po = okt.pos(row)    
    for k in range(len(po)):
      if po[k][1] == 'Verb':
        v = v+","+po[k][0]
      if po[k][1] == "Adjective":
        adj = adj+","+po[k][0]
    dataframe.at[i,'verb'] = v
    dataframe.at[i,'adjective'] = adj
  return dataframe

In [ ]:
tokenizer(df,'split')

,App Name,Rating,Comment,split,advertisement,connect,buying,fun,pos,verb,adjective
0,오딘: 발할라 라이징,1,첫날에 결제를 하고 꾸준히 소소하게 즐기려고 했으나 대기열이 많아 접속자체를 할수가...,첫날에 결제를 하고 꾸준히 대기 열이 많아 접속 자체를 할 수가 없습니다.,0,1,1,0,",하고,꾸준히,많아,할,없습니다",",하고,할",",꾸준히,많아,없습니다"
1,오딘: 발할라 라이징,1,첫날에 결제를 하고 꾸준히 소소하게 즐기려고 했으나 대기열이 많아 접속자체를 할수가...,돈은 썼는데 게임을 하다가 질려서 접은 것도 아니고 접속이 돼야 무슨 게임을 하죠,0,1,0,0,",썼는데,하다가,질려서,접은,아니고,돼야,하죠",",썼는데,하다가,질려서,접은,돼야,하죠",",아니고"
2,오딘: 발할라 라이징,1,첫날에 결제를 하고 꾸준히 소소하게 즐기려고 했으나 대기열이 많아 접속자체를 할수가...,하루 종일 대기 열기 다리느라 핸드폰을 켜 둘 수도 없고 직장생활하며 집에서 잠깐씩...,0,1,0,0,",다리느라,없고,하려는데,되나요",",다리느라,하려는데,되나요",",없고"
3,오딘: 발할라 라이징,1,첫날에 결제를 하고 꾸준히 소소하게 즐기려고 했으나 대기열이 많아 접속자체를 할수가...,대기 열을 계속 이대로 둘 거면 환불해주시기 바랍니다.,0,0,0,0,",해주시기,바랍니다",",해주시기,바랍니다",
4,오딘: 발할라 라이징,1,모바일 게임 그동안 참 많이 했는데... 다른 게임들은 그나마 실행후 플레이 하면서...,모바일 게임 그동안 참 많이 했는데...,0,0,0,0,",참,했는데",",참,했는데",
...,...,...,...,...,...,...,...,...,...,...,...
33176,Rider,1,응 개똥게임 차선택이안된고클릭도 안되,응 개똥 게임 찬 선택이 안된고 클릭도 안돼,0,1,0,0,",안된고,돼",",돼",",안된고"
33177,Rider,5,눈동자 지 너무 잘 먹고,눈동자 지 너무 잘 먹고,0,0,0,0,",지,잘,먹고",",지,잘,먹고",
33178,Rider,5,그냥그대로멈추는데? 와 버그대박 불상해서 많ㅇㄱ드릴게요,그냥 그대로 멈추는데?,0,0,0,0,",멈추는데",",멈추는데",
33179,Rider,5,그냥그대로멈추는데? 와 버그대박 불상해서 많ㅇㄱ드릴게요,와 버그 대박 불상해서 많ㅇㄱ드릴게요,0,1,0,0,",와,해서,많,드릴게요",",와,해서,드릴게요",",많"


In [ ]:
df.to_csv('/content/drive/MyDrive/munning_rachine/dataset/keyword/PlaystoreReviewData_preidcate.csv', encoding='utf-8-sig', index=False)

#### word2vec 써보기

In [ ]:
# 동사를 ,단위로 잘라서 최종적으로 이중 리스트로 만듦
verb_list = df['verb'].map(lambda x : x.split(',')[1:])
adj_list = df['adjective'].map(lambda x : x.split(',')[1:])

In [ ]:
# 동사가 없는 줄을 제외
verb_list = [v for v in verb_list if len(v) >= 1]
adj_list = [v for v in adj_list if len(v) >= 1]
verb_list[:10]

[['하고', '할'],
 ['썼는데', '하다가', '질려서', '접은', '돼야', '하죠'],
 ['다리느라', '하려는데', '되나요'],
 ['해주시기', '바랍니다'],
 ['참', '했는데'],
 ['하면서'],
 ['해보는', '따기', '타다', '보내고', '할', '해서', '하더라도', '끝나는', '개면', '보다'],
 ['떨어지고', '하는', '합니다'],
 ['보면', '따라', '하다'],
 ['보다', '하다는', '들을']]

In [ ]:
from gensim.models.word2vec import Word2Vec

model = Word2Vec(verb_list, sg=1, window=5, min_count=1)
model.init_sims(replace=True)

In [ ]:
model.wv.similar_by_word('하다', 100)

[('들어가', 0.9943310022354126),
 ('보니', 0.9940982460975647),
 ('되는', 0.9940873980522156),
 ('잘리는', 0.9939504265785217),
 ('들어', 0.9937570095062256),
 ('생겼으면', 0.9937509298324585),
 ('모으기', 0.9937261343002319),
 ('들어가지', 0.993708610534668),
 ('없어지고', 0.993635356426239),
 ('보다', 0.9936319589614868),
 ('들어가지고', 0.9936276078224182),
 ('쓰고', 0.9935319423675537),
 ('올라갈', 0.9935200810432434),
 ('모으', 0.9935113191604614),
 ('걸려서', 0.9935055375099182),
 ('알았는데', 0.993502676486969),
 ('주고', 0.9934976696968079),
 ('납니다', 0.9934850931167603),
 ('하게', 0.9934848546981812),
 ('되고요', 0.9934748411178589),
 ('쓰는', 0.9934535026550293),
 ('만들지', 0.9934502840042114),
 ('나네요', 0.993445634841919),
 ('보이는데', 0.9934338927268982),
 ('깨는', 0.993429958820343),
 ('나오는데', 0.9934231042861938),
 ('따라', 0.9934161305427551),
 ('하다가', 0.9934155941009521),
 ('알겠는데', 0.9934132695198059),
 ('나지', 0.9934118986129761),
 ('됨', 0.9934114217758179),
 ('걸리고', 0.9934061765670776),
 ('해주는', 0.9933923482894897),
 ('속이', 0.9933828711

#### 상위 100 키워드 추출 (동사, 형용사)

In [ ]:
#이중 리스트를 하나의 리스트로
verb_list_1d = sum(verb_list, [])
adj_list_1d = sum(adj_list, [])

In [ ]:
from collections import Counter
verb_top100 = Counter(verb_list_1d).most_common(100)
adj_top100 = Counter(adj_list_1d).most_common(100)

In [ ]:
verb_top100 = pd.DataFrame(verb_top100, columns=['동사','동사_빈도수'])
verb_top100

,동사,동사_빈도수
0,할,2131
1,하는,1802
2,한,1148
3,하면,1125
4,하고,1085
...,...,...
95,줄여주세요,94
96,해봤는데,93
97,했어요,93
98,걸려서,93


In [ ]:
adj_top100 = pd.DataFrame(adj_top100, columns=['형용사','형용사_빈도수'])
adj_top100

,형용사,형용사_빈도수
0,입니다,1524
1,있는,999
2,같아요,729
3,재밌어요,688
4,좋아요,678
...,...,...
95,안됩니다,80
96,없어,80
97,있고요,79
98,있었으면,79


In [ ]:
pd.concat([verb_top100, adj_top100],axis=1).to_csv('/content/drive/MyDrive/munning_rachine/dataset/keyword/verb_adj_top100.csv', encoding='utf-8-sig', index=False)

#### 감성 사전을 이용하여 점수화 하기


In [ ]:
senti_df = pd.read_csv("/content/drive/MyDrive/munning_rachine/dataset/keyword/verb_adj_top100.csv")
senti_df

,pos,freq,senti
0,할,2131,0
1,하는,1802,0
2,한,1148,0
3,하면,1125,0
4,하고,1085,0
...,...,...,...
195,안됩니다,80,-1
196,없어,80,-1
197,있고요,79,0
198,있었으면,79,-1


In [ ]:
senti = senti_df[['pos', 'senti']]
senti = senti[senti['senti'] != 0]

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/munning_rachine/dataset/PlaystoreData.csv')

In [ ]:
# 동일한 게임의 리뷰끼리 각 네 가지 항목의 동사 및 형용사의 긍정/부정 지수를 측정한다.
# 지수는 세자리수에서 반올림
def make_senti_score(data, df, col_name):
  for (i, app_name) in enumerate(data['App Name']):
    cnt, pos_scr, neg_scr = 0, 0, 0
    for sentence in df[(df[col_name] == 1) & (df['App Name'] == app_name)]['split']:
      cnt += 1
      for (j, pos) in enumerate(senti['pos']):
        if pos in sentence:
          temp_scr = senti['senti'].iloc[j]
          if temp_scr > 0:
            pos_scr += temp_scr
          else:
            neg_scr += temp_scr 

    data.at[i, col_name+'_cnt'] = cnt
    data.at[i, col_name+'_pos_scr'] = pos_scr
    data.at[i, col_name+'_neg_scr'] = neg_scr
    try:
      data.at[i, col_name+'_pos_senti'] = round(pos_scr / cnt, 3) * 100
      data.at[i, col_name+'_neg_senti'] = -(round(neg_scr / cnt, 3)) * 100
      data.at[i, col_name+'_neu_senti'] =  round((cnt-pos_scr+neg_scr) / cnt, 3) * 100 
    except ZeroDivisionError:
      data.at[i, col_name+'_pos_senti'] = 0
      data.at[i, col_name+'_neg_senti'] = 0
      data.at[i, col_name+'_neu_senti'] = 0
  return data

In [ ]:
data = make_senti_score(data, df, 'advertisement')
data = make_senti_score(data, df, 'connect')
data = make_senti_score(data, df, 'buying')
data = make_senti_score(data, df, 'fun')

In [ ]:
data

,Collection Date,App Name,App Developer,App Category,App Pop Rank,App Download,App Review Count,App Charge,App Limit,advertisement_cnt,advertisement_pos_scr,advertisement_neg_scr,advertisement_pos_senti,advertisement_neg_senti,advertisement-neu_senti,advertisement_neu_senti,connect_cnt,connect_pos_scr,connect_neg_scr,connect_pos_senti,connect_neg_senti,connect-neu_senti,connect_neu_senti,buying_cnt,buying_pos_scr,buying_neg_scr,buying_pos_senti,buying_neg_senti,buying-neu_senti,buying_neu_senti,fun_cnt,fun_pos_scr,fun_neg_scr,fun_pos_senti,fun_neg_senti,fun-neu_senti,fun_neu_senti
0,2021-07-05 21:30,오딘: 발할라 라이징,Kakao Games Corp.,롤플레잉,1,"500,000+","60,834",Free,18세 이상,6.0,0.0,-4.0,0.0,66.7,33.3,33.3,156.0,14.0,-76.0,9.0,48.7,42.3,42.3,152.0,24.0,-81.0,15.8,53.3,30.9,30.9,36.0,13.0,-16.0,36.1,44.4,19.4,19.4
1,2021-07-05 21:30,히어로즈 테일즈,37 Mobile Games,롤플레잉,2,"100,000+","1,749",Free,12세 이상,13.0,5.0,-11.0,38.5,84.6,-23.1,-23.1,58.0,8.0,-34.0,13.8,58.6,27.6,27.6,46.0,14.0,-21.0,30.4,45.7,23.9,23.9,38.0,26.0,-14.0,68.4,36.8,-5.3,-5.3
2,2021-07-05 21:30,Brain Rush - Brain Hole Bang,MNMFUN,퍼즐,3,"1,000,000+","8,542",Free,15세 이상,94.0,18.0,-42.0,19.1,44.7,36.2,36.2,20.0,7.0,-12.0,35.0,60.0,5.0,5.0,8.0,3.0,-4.0,37.5,50.0,12.5,12.5,26.0,17.0,-9.0,65.4,34.6,0.0,0.0
3,2021-07-05 21:30,Bubble Ouch: Pop it Fidgets & Bubble Wrap Game,WannaPlay LLC,캐주얼 게임,4,"1,000,000+",964,Free,전체이용가,28.0,5.0,-16.0,17.9,57.1,25.0,25.0,1.0,1.0,0.0,100.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,7.0,5.0,-4.0,71.4,57.1,-28.6,-28.6
4,2021-07-05 21:30,FNF music battle:Girlfriend’s Ex-Tabi vs Boyfr...,tung son,음악,5,"100,000+","1,473",Free,전체이용가,11.0,3.0,-8.0,27.3,72.7,0.0,0.0,7.0,3.0,-4.0,42.9,57.1,0.0,0.0,20.0,7.0,-11.0,35.0,55.0,10.0,10.0,39.0,22.0,-14.0,56.4,35.9,7.7,7.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2021-07-05 21:30,고양이 가구점 : 냥냥 타워 만들기,"mafgames (Idle Games, Tycoon Games)",시뮬레이션,96,"100,000+","14,452",Free,전체이용가,83.0,13.0,-37.0,15.7,44.6,39.8,39.8,69.0,9.0,-26.0,13.0,37.7,49.3,49.3,123.0,15.0,-49.0,12.2,39.8,48.0,48.0,25.0,9.0,-8.0,36.0,32.0,32.0,32.0
96,2021-07-05 21:30,타일 연결 - 무료 타일 퍼즐 & 매치 브레인 게임,Higgs Studio,보드,97,"10,000,000+","133,269",Free,전체이용가,103.0,31.0,-60.0,30.1,58.3,11.7,11.7,12.0,0.0,-4.0,0.0,33.3,66.7,66.7,8.0,3.0,-1.0,37.5,12.5,50.0,50.0,29.0,22.0,-8.0,75.9,27.6,-3.4,-3.4
97,2021-07-05 21:30,FIFA Mobile,NEXON Company,스포츠,98,"1,000,000+","48,501",Free,전체이용가,0.0,0.0,0.0,0.0,0.0,NaN,0.0,134.0,21.0,-88.0,15.7,65.7,18.7,18.7,124.0,27.0,-63.0,21.8,50.8,27.4,27.4,30.0,17.0,-9.0,56.7,30.0,13.3,13.3
98,2021-07-05 21:30,메이크업 스튜디오 3D,Kwalee Ltd,캐주얼 게임,99,"5,000,000+","36,113",Free,전체이용가,112.0,31.0,-73.0,27.7,65.2,7.1,7.1,27.0,7.0,-20.0,25.9,74.1,0.0,0.0,4.0,0.0,-1.0,0.0,25.0,75.0,75.0,39.0,18.0,-12.0,46.2,30.8,23.1,23.1


In [ ]:
data.to_csv('/content/drive/MyDrive/munning_rachine/dataset/PlaystoreData_final.csv', encoding='utf-8-sig', index=False)

In [ ]:
df['App Name'].unique()

array(['오딘: 발할라 라이징', '히어로즈 테일즈', 'Brain Rush - Brain Hole Bang',
       'Bubble Ouch: Pop it Fidgets & Bubble Wrap Game',
       'FNF music battle:Girlfriend’s Ex-Tabi vs Boyfriend', 'X2: 이클립스',
       '비홍지경', '소녀X헌터', 'Fidget Toys 3D - Fidget Cube, AntiStress & Calm',
       '블라스트M', 'Kaiju Run', '마스터 카운트: 군중 클래시 & 스틱맨 달리기 게임', '무한의 계단',
       'Super Pop It Simulator: ASMR Relax', '한국밈 피하기', '마이리틀포레스트',
       'Paper Fold', '퍼즐 오브 Z', '냥코 대전쟁', '지방이 적합!',
       '제2의 나라: CROSS WORLDS', 'Undertale but FNF gameplay', '행복한 색칠나라',
       '매직 타일 3', '쿠키런: 오븐브레이크 - 쿠키들의 러닝 게임!', 'Tape Thrower',
       'Hair Challenge', '쿠키런: 킹덤', '시크릿 쥬쥬 : 쥬쥬 코디 놀이',
       'Music Fighter Bomb head guy', 'ROBLOX',
       '워드퍼즐 - 단어 게임! 재미있는 무료 단어 퍼즐', '백야극광',
       '스시 롤 3D (Sushi Roll 3D) - 음식 게임 ASMR', 'Among Us',
       '마이티 파워: 배틀 히어로즈. 매직 레전드.',
       'Craft World - Master Building Block Game 3D', '피쉬돔 (Fishdom)',
       'Like A Dino!', '건담 슈프림 배틀', '천상나르샤',
       'FNF Mod Garcello, Tricky, Satoru

In [ ]:
d = list(df[(df['App Name'] =='브롤스타즈') & (df['buying'] == 0) & (df['connect'] == 0) & (df['fun'] == 1) & (df['advertisement'] == 0)]['split'])
# d = list(df[(df['App Name'] =='오딘: 발할라 라이징') & (df['connect'] == 1)]['split'])
" ".join(d)

'아~주 재미있습니다! 고의적으로 게임하는 사람들 때문에 스트레스 너무 많이 받아요ㅠ... 다른 게임은 다 지워도 이것만큼은 지운 적이 없는데.. ㅜ.. 일단 정말 재미있습니다! 그래도 재밌으니까 한번 해보는 걸 추천해요! 신규 맵 도 많이 나오고 신규 패스랑 신규 크롤러까지 이렇게 재미있는 게임을 만들어줘서 감사하고 앞으로도 재밌게 합시다 파이팅! 아 그리고 점수도 5점 드립니다 고맙습니다 순수 재미를 아주 잘 공략한 게임입니다. 실력 게임이라 차이가 별로 안 나요. 장점 1. 사람이랑 해서 재미있음 3. 재미있음 재미있어요!! 그런데 저희 팀 중 한 명은 5520 그리고 다른 한 명은 12118 네 이분은 저랑 차이가 별로 나진 않았습니다. 정말 재미있는데 레드 와이파이 좀 고치면 좋켔어요. 장점들 재미있다, 프롤 스타즈는 정말 재미있는 게임입니다 3재미 있오. 재미있지만 몇 가지 고쳐야 할 점이 있습니다 일단 게임 자체는 너무 재미있어요 하지만 게임은 재미있어서 별 5개 드립니다^^ 이거 빼면 다 재미있고 심심할 때 해도 좋고 그러니깐 강추예요 저 원래 게임을 별로 안 하는데 이 게임은 진짜 계속하게 돼요. 게임은 재미있습니다 재미있어요! 프롤 하는 거 정말 추천합니다!!! 그래도 재미있으니 싸 별점 4점 드리도록 하겠습니다. 너무 재미있습니다. 앞으로 더 게임 재미있게 바꿔 주십쇼!! 너무 재미있는데 레드 와이파이 때문에 움직이지도 못하고 공격도 못해요 게꿀잼 추천!!!!! 슈퍼셀 짱 프롤 재미있어요 재미있어요 중독성도 꽤 있고 맵 메이 거도 정말 꿀 잼이네요. 옛날에 친구가 추천해서 했었는데 지금은 정말 심하네요. 너무나도 재미있습니다. 너무 재미있는 게임이에요!! 하지만 너무 재미있고 좋은 게임이라 생각합니다!! 이건 최고 아니 아니 최고보다 더 좋은 말에 최고인 말이 있으면 좋을 것 같네요 게임은 대체로 재미있고 밸런스도 좋습니다. 이 게임은 너무나도 재미있는 게임입니다. 하지만 재미있습니다. 이 게임 강력 추천! 그리고 그 롤러도 다 마음에 들어

## 추천 시스템

In [ ]:
pdata = pd.read_csv('/content/drive/MyDrive/munning_rachine/dataset/게임_키워드별점수.csv', encoding='utf-8')

In [ ]:
pdata.head()

,App Name,advertisement_pos_senti,advertisement_neg_senti,connect_pos_senti,connect_neg_senti,buying_pos_senti,buying_neg_senti,fun_pos_senti,fun_neg_senti
0,오딘: 발할라 라이징,0.0,66.7,9.0,48.7,15.8,53.3,36.1,44.4
1,히어로즈 테일즈,38.5,84.6,13.8,58.6,30.4,45.7,68.4,36.8
2,Brain Rush - Brain Hole Bang,19.1,44.7,35.0,60.0,37.5,50.0,65.4,34.6
3,Bubble Ouch: Pop it Fidgets & Bubble Wrap Game,17.9,57.1,100.0,0.0,0.0,0.0,71.4,57.1
4,FNF music battle:Girlfriend’s Ex-Tabi vs Boyfr...,27.3,72.7,42.9,57.1,35.0,55.0,56.4,35.9


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
pdata.set_index('App Name')
pdata2 = pdata['App Name']
pdata = pdata.drop('App Name', axis = 1)

,advertisement_pos_senti,advertisement_neg_senti,connect_pos_senti,connect_neg_senti,buying_pos_senti,buying_neg_senti,fun_pos_senti,fun_neg_senti
0,0.0,66.7,9.0,48.7,15.8,53.3,36.1,44.4
1,38.5,84.6,13.8,58.6,30.4,45.7,68.4,36.8
2,19.1,44.7,35.0,60.0,37.5,50.0,65.4,34.6
3,17.9,57.1,100.0,0.0,0.0,0.0,71.4,57.1
4,27.3,72.7,42.9,57.1,35.0,55.0,56.4,35.9
...,...,...,...,...,...,...,...,...
95,15.7,44.6,13.0,37.7,12.2,39.8,36.0,32.0
96,30.1,58.3,0.0,33.3,37.5,12.5,75.9,27.6
97,0.0,0.0,15.7,65.7,21.8,50.8,56.7,30.0
98,27.7,65.2,25.9,74.1,0.0,25.0,46.2,30.8


In [ ]:
similarity_simple_pair = cosine_similarity(pdata, pdata)
cos_pair = pd.DataFrame(similarity_simple_pair)
cos_pair

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,1.000000,0.930197,0.902011,0.575678,0.932348,0.772679,0.708256,0.979899,0.752548,0.923136,0.877842,0.880513,0.899953,0.598183,0.914863,0.800482,0.717291,0.940578,0.747299,0.938067,0.993396,0.570369,0.775278,0.911964,0.970864,0.494197,0.967086,0.498239,0.792371,0.611499,0.868366,0.922994,0.735360,0.861081,0.786028,0.960223,0.738894,0.901497,0.803454,0.968797,...,0.237764,0.793155,0.956784,0.901167,0.912919,0.906624,0.879221,0.902057,0.869661,0.907887,0.847757,0.952172,0.939625,0.625688,0.747891,0.926180,0.835904,0.336858,0.695017,0.901940,0.913163,0.841560,0.748296,0.935807,0.917207,0.958833,0.657119,0.702479,0.831050,0.845795,0.973822,0.918427,0.989103,0.328501,0.698290,0.974425,0.802144,0.783207,0.893586,0.917997
1,0.930197,1.000000,0.941334,0.642165,0.967358,0.745754,0.722471,0.909563,0.746720,0.904623,0.927844,0.923841,0.946451,0.631627,0.949176,0.747096,0.900014,0.910324,0.926824,0.939910,0.916984,0.704878,0.930102,0.935449,0.904039,0.509585,0.926046,0.710223,0.858916,0.730978,0.977976,0.984372,0.868109,0.943220,0.729444,0.944660,0.802590,0.985962,0.903010,0.947175,...,0.527299,0.752149,0.887283,0.917839,0.854627,0.893346,0.965058,0.962439,0.942283,0.981802,0.887114,0.901455,0.904648,0.657091,0.761352,0.961957,0.873481,0.505369,0.849716,0.994112,0.910062,0.896547,0.909675,0.956343,0.937686,0.908327,0.738604,0.893981,0.948309,0.945415,0.951968,0.970251,0.941833,0.613798,0.880025,0.972113,0.941076,0.760992,0.938606,0.941387
2,0.902011,0.941334,1.000000,0.683925,0.975525,0.884984,0.894871,0.830656,0.815998,0.965134,0.896135,0.947444,0.977871,0.766528,0.988940,0.873633,0.871796,0.947710,0.853398,0.895756,0.875482,0.826797,0.896719,0.937730,0.942863,0.549343,0.933560,0.756293,0.830683,0.612638,0.891407,0.982072,0.865932,0.958719,0.866567,0.933715,0.795033,0.938850,0.894846,0.893076,...,0.482120,0.893542,0.883353,0.943841,0.925209,0.950340,0.931156,0.884083,0.925940,0.947193,0.729766,0.908401,0.899689,0.821415,0.910587,0.962769,0.938215,0.584346,0.907119,0.928971,0.917677,0.888476,0.907186,0.970173,0.924727,0.931368,0.790462,0.812853,0.950008,0.962726,0.912945,0.962038,0.934076,0.532720,0.864613,0.955729,0.877893,0.903327,0.899870,0.919605
3,0.575678,0.642165,0.683925,1.000000,0.719087,0.587739,0.622119,0.599955,0.357775,0.514848,0.574574,0.779624,0.602379,0.779830,0.689621,0.443893,0.713774,0.593345,0.706550,0.645334,0.607600,0.544735,0.710703,0.646186,0.626021,0.613448,0.538995,0.549103,0.521900,0.617109,0.662463,0.649017,0.647530,0.767103,0.536328,0.533261,0.593861,0.594437,0.685653,0.653096,...,0.444424,0.441577,0.529028,0.742182,0.663811,0.766094,0.618224,0.691751,0.503436,0.671445,0.575147,0.471566,0.587780,0.422995,0.605367,0.660380,0.547943,0.401024,0.714377,0.653585,0.440581,0.500327,0.691603,0.561164,0.676127,0.538476,0.332921,0.627500,0.604507,0.607633,0.582056,0.629978,0.598450,0.501647,0.569951,0.648438,0.631231,0.456938,0.656568,0.537892
4,0.932348,0.967358,0.975525,0.719087,1.000000,0.790229,0.800586,0.896090,0.801894,0.929958,0.927321,0.946006,0.976972,0.678422,0.974326,0.805962,0.862918,0.922387,0.899412,0.921624,0.915582,0.787431,0.893054,0.941612,0.930797,0.463227,0.943069,0.717020,0.836003,0.637848,0.947871,0.973804,0.884508,0.952433,0.782726,0.927606,0.781462,0.960459,0.942509,0.943691,...,0.429917,0.799547,0.901927,0.957497,0.921353,0.961668,0.948811,0.958249,0.922078,0.983751,0.839362,0.914639,0.920821,0.732105,0.805632,0.945910,0.927191,0.470861,0.886452,0.964575,0.907504,0.878466,0.915782,0.954342,0.954523,0.946011,0.779680,0.844840,0.926138,0.955943,0.941312,0.969885,0.945134,0.507558,0.859152,0.973649,0.869481,0.803098,0.926327,0.933525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [ ]:
game_similarity = pd.DataFrame(
    data=similarity_simple_pair,
    index=pdata2,
    columns=pdata2
)

In [ ]:
game_similarity

App Name,오딘: 발할라 라이징,히어로즈 테일즈,Brain Rush - Brain Hole Bang,Bubble Ouch: Pop it Fidgets & Bubble Wrap Game,FNF music battle:Girlfriend’s Ex-Tabi vs Boyfriend,X2: 이클립스,비홍지경,소녀X헌터,"Fidget Toys 3D - Fidget Cube, AntiStress & Calm",블라스트M,Kaiju Run,마스터 카운트: 군중 클래시 & 스틱맨 달리기 게임,무한의 계단,Super Pop It Simulator: ASMR Relax,한국밈 피하기,마이리틀포레스트,Paper Fold,퍼즐 오브 Z,냥코 대전쟁,지방이 적합!,제2의 나라: CROSS WORLDS,Undertale but FNF gameplay,행복한 색칠나라,매직 타일 3,쿠키런: 오븐브레이크 - 쿠키들의 러닝 게임!,Tape Thrower,Hair Challenge,쿠키런: 킹덤,시크릿 쥬쥬 : 쥬쥬 코디 놀이,Music Fighter Bomb head guy,ROBLOX,워드퍼즐 - 단어 게임! 재미있는 무료 단어 퍼즐,백야극광,스시 롤 3D (Sushi Roll 3D) - 음식 게임 ASMR,Among Us,마이티 파워: 배틀 히어로즈. 매직 레전드.,Craft World - Master Building Block Game 3D,피쉬돔 (Fishdom),Like A Dino!,건담 슈프림 배틀,...,Brain Riddle - Tricky Puzzles,리그 오브 레전드: 와일드 리프트,Bridge Race,Cross Logic - 두뇌 퍼즐 게임,Brain Puzzle:Tricky IQ Riddle‪s,사커 슈퍼스타(Soccer Super Star),숨겨진 유산,행복의 저택,Rummikub,지오메트리 대시 라이트 (Geometry Dash L),Hunting Clash: 동물사냥게임. 슈팅 시뮬레이터,Ball Run 2048,Going Balls,달토끼 키우기,Pokémon GO,로데오 스템피드: 스카이 동물원 사파리,Catwalk Beauty,Horror FNF music battle: BF vs Tricky,숫자로 색칠하기 - 무료 색칠 공부 및 퍼즐 게임,캔디크러쉬사가,Woodoku,Run of Life,Tiles Hop: EDM Rush!,Heroes Inc!,Project Makeover,Tile Master,"ASolver(에이솔버) - 퍼즐을 보여 줘, 내가 풀어내겠어!",카트라이더 러쉬플러스,스도쿠 - 무료 두뇌훈련 스도쿠 퍼즐,아스팔트 9: 레전드,꿈의 정원 (Gardenscapes),GIRL GLOBE,파이널삼국지2,Star Art: Drawing & Relaxing,스펠마스터 : 실시간 마법 디펜스 대전,고양이 가구점 : 냥냥 타워 만들기,타일 연결 - 무료 타일 퍼즐 & 매치 브레인 게임,FIFA Mobile,메이크업 스튜디오 3D,Rider
App Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
오딘: 발할라 라이징,1.000000,0.930197,0.902011,0.575678,0.932348,0.772679,0.708256,0.979899,0.752548,0.923136,0.877842,0.880513,0.899953,0.598183,0.914863,0.800482,0.717291,0.940578,0.747299,0.938067,0.993396,0.570369,0.775278,0.911964,0.970864,0.494197,0.967086,0.498239,0.792371,0.611499,0.868366,0.922994,0.735360,0.861081,0.786028,0.960223,0.738894,0.901497,0.803454,0.968797,...,0.237764,0.793155,0.956784,0.901167,0.912919,0.906624,0.879221,0.902057,0.869661,0.907887,0.847757,0.952172,0.939625,0.625688,0.747891,0.926180,0.835904,0.336858,0.695017,0.901940,0.913163,0.841560,0.748296,0.935807,0.917207,0.958833,0.657119,0.702479,0.831050,0.845795,0.973822,0.918427,0.989103,0.328501,0.698290,0.974425,0.802144,0.783207,0.893586,0.917997
히어로즈 테일즈,0.930197,1.000000,0.941334,0.642165,0.967358,0.745754,0.722471,0.909563,0.746720,0.904623,0.927844,0.923841,0.946451,0.631627,0.949176,0.747096,0.900014,0.910324,0.926824,0.939910,0.916984,0.704878,0.930102,0.935449,0.904039,0.509585,0.926046,0.710223,0.858916,0.730978,0.977976,0.984372,0.868109,0.943220,0.729444,0.944660,0.802590,0.985962,0.903010,0.947175,...,0.527299,0.752149,0.887283,0.917839,0.854627,0.893346,0.965058,0.962439,0.942283,0.981802,0.887114,0.901455,0.904648,0.657091,0.761352,0.961957,0.873481,0.505369,0.849716,0.994112,0.910062,0.896547,0.909675,0.956343,0.937686,0.908327,0.738604,0.893981,0.948309,0.945415,0.951968,0.970251,0.941833,0.613798,0.880025,0.972113,0.941076,0.760992,0.938606,0.941387
Brain Rush - Brain Hole Bang,0.902011,0.941334,1.000000,0.683925,0.975525,0.884984,0.894871,0.830656,0.815998,0.965134,0.896135,0.947444,0.977871,0.766528,0.988940,0.873633,0.871796,0.947710,0.853398,0.895756,0.875482,0.826797,0.896719,0.937730,0.942863,0.549343,0.933560,0.756293,0.830683,0.612638,0.891407,0.982072,0.865932,0.958719,0.866567,0.933715,0.795033,0.938850,0.894846,0.893076,...,0.482120,0.893542,0.883353,0.943841,0.925209,0.950340,0.931156,0.884083,0.925940,0.947193,0.729766,0.908401,0.899689,0.821415,0.910587,0.962769,0.938215,0.584346,0.907119,0.928971,0.917677,0.888476,0.907186,0.970173,0.924727,0.931368,0.790462,0.812853,0.950008,0.962726,0.912945,0.962038,0.934076,0.532720,0.864613,0.955729,0.877893,0.903327,0.899870,0.919605
Bubble Ouch: Pop it Fidgets & Bubble Wrap Game,0.575678,0.642165,0.683925,1.000000,0.719087,0.587739,0.622119,0.599955,0.357775,0.514848,0.574574,0.779624,0.602379,0.779830,0.689621,0.443893,0.713774,0.593345,0.706550,0.645334,0.6076

In [12]:
import pandas as pd
game_similarity = pd.read_csv('/content/drive/MyDrive/munning_rachine/dataset/게임_코사인유사도.csv', encoding='utf-8-sig', index_col='App Name')

In [26]:
def recommand(game):
  return game_similarity[game].sort_values(ascending=False)[1:4]

In [28]:
pd.DataFrame(recommand('ROBLOX'))

In [ ]:
game_similarity.to_csv('/content/drive/MyDrive/munning_rachine/dataset/게임_코사인유사도.csv', encoding='utf-8-sig')